In [57]:
import pandas as pd
import datetime

#number of line to use in each file (possible improvment : use Panda's chunk to process all the data)
LIMIT_B = 50000
LIMIT_S = 3000
#searches dataset
dfS = pd.read_csv("searches.csv", sep='^', nrows=LIMIT_S,usecols=['Date','NbSegments','Seg1Departure','Seg2Departure','Seg3Departure','Seg1Date','Seg2Date','Seg3Date','Seg1Arrival','Seg2Arrival','Seg3Arrival'])
#booking dataset
dfB = pd.read_csv("bookings.csv", sep='^', nrows=LIMIT_B,usecols=['brd_port','off_port','brd_time           ','cre_date           '])
#dataset use to create the final csv
dfEnd = pd.read_csv("searches.csv", sep='^', nrows=LIMIT_S)

#number of line
row_countS = sum(1 for line in open("searches.csv"))
print("number of lines in searches.csv :"+str(row_countS))
row_countB = sum(1 for line in open("bookings.csv"))
print("number of lines in bookings.csv :"+str(row_countB))

#Clear some spaces in the data
dfB['brd_port'] = dfB['brd_port'].str.strip()
dfB['off_port'] = dfB['off_port'].str.strip()

#we only need the yy/mm/dd of cre_date to compare with Date, moreover cre_date time is always 00:00:00
dfB['cre_date           '] = dfB['cre_date           '].apply(lambda x: x.split(' ')[0]).str.strip()
#we assumed cre_date is set to d+1:00:00:00 so we take the previous date to compare cre_time and searche date
dfB['cre_date           '] = dfB['cre_date           '].apply(lambda x: str(datetime.date(int(x.split('-')[0]), int(x.split('-')[1]), int(x.split('-')[2])) - datetime.timedelta(days=1))).str.strip()


#We want to keep only the yy/mm/dd of boarding time in booking to compare with Seg_x_Date
dfB['brd_time           '] = dfB['brd_time           '].apply(lambda x: x.split(' ')[0]).str.strip()



from itertools import islice


def recherche(data):
        origin = data['Seg1Departure']
        destination = data['Seg1Arrival']
        date = data['Seg1Date']
        dateCrea = data['Date']
        #should always be true
        if isinstance(date, str):
            #The creation time of the booking should be the same (we did day-1 to realign it) as the date of the search boarding time of a booking should match with Seg1Departure, same for Seg1Departure/Boarding port and Seg1Arrival/destination port
            mergeS1 = dfB.loc[((dfB['cre_date           '] == dateCrea) & (dfB['brd_time           '] == date) & (dfB['brd_port'] == origin) & (dfB['off_port'] == destination))     ]
            #at least one booking match the criteria of the searche
            if not mergeS1.empty:
                date2 = data['Seg2Date']
                origin2 = data['Seg2Departure']
                destination2 = data['Seg2Arrival']
                #the searche has at least 2 segments, we will look if another booking match the segment
                if isinstance(date2, str):
                    #The creation date of the booking should also be the same, this time we compare boarding/origine/../destination with segment2
                    mergeS2 = dfB.loc[(dfB['cre_date           '] == dateCrea) & (dfB['brd_time           '] == date2) & (dfB['brd_port'] == origin2) & (dfB['off_port'] == destination2)     ]
                    #at least one booking match the criteria of the searche for segment 1 and 2
                    if not mergeS2.empty:
                        date3 = data['Seg3Date']
                        origin3 = data['Seg3Departure']
                        destination3 = data['Seg3Arrival']                   
                        #the searche has at least 3 segments, we will look if another booking match the segment
                        if isinstance(date3, str):
                            #This time we use segment3
                            mergeS3 = dfB.loc[(dfB['cre_date           '] == dateCrea) & (dfB['brd_time           '] == date3) & (dfB['brd_port'] == origin3) & (dfB['off_port'] == destination3)     ]
                            #at least one booking match the criteria of the searche for segment 1 to 3
                            if not mergeS3.empty:
                                date4 = data['Seg4Date']
                                origin4 = data['Seg4Departure']
                                destination4 = data['Seg4Arrival'] 
                                #the searche has at least 4 segments, we will look if another booking match the segment
                                if isinstance(date4, str):
                                    #This time we use segment4
                                    mergeS4 = dfB.loc[(dfB['cre_date           '] == dateCrea) & (dfB['brd_time           '] == date4) & (dfB['brd_port'] == origin4) & (dfB['off_port'] == destination4)     ]
                                    #at least one booking match the criteria of the searche for segment 1 to 4
                                    if not mergeS4.empty:
                                        date5 = data['Seg5Date']
                                        origin5 = data['Seg5Departure']
                                        destination5 = data['Seg5Arrival'] 
                                        #the searche has at least 5 segments, we will look if another booking match the segment
                                        if isinstance(date5, str):
                                            #This time we use segment5
                                            mergeS5 = dfB.loc[(dfB['cre_date           '] == dateCrea) & (dfB['brd_time           '] == date5) & (dfB['brd_port'] == origin5) & (dfB['off_port'] == destination5)     ]
                                            #at least one booking match the criteria of the searche for segment 1 to 5
                                            if not mergeS5.empty:
                                                date6 = data['Seg6Date']
                                                origin6 = data['Seg6Departure']
                                                destination6 = data['Seg6Arrival'] 
                                                #the searche has 6 segments, we will look if another booking match the segment
                                                if isinstance(date6, str):
                                                    #This time we use segment6
                                                    mergeS6 = dfB.loc[(dfB['cre_date           '] == dateCrea) & (dfB['brd_time           '] == date6) & (dfB['brd_port'] == origin6) & (dfB['off_port'] == destination6)     ]
                                                    #at least one booking match the criteria of the searche for segment 1 to 6
                                                    if not mergeS6.empty:
                                                        #every segments match
                                                        return '1'
                                                    #segment 6 does not match a booking
                                                    else:
                                                        return '0'
                                                #the searche is a five segments search and we found at least one booking that match      
                                                else:
                                                    return '1'
                                            #segment 5 does not match a booking
                                            else:
                                                return '0'
                                        #the searche is a four segments search and we found at least one booking that match      
                                        else:
                                            return '1'
                                    #segment 4 does not match a booking
                                    else:
                                        return '0'
                                #the searche is a three segments search and we found at least one booking that match      
                                else:
                                    return '1'
                            #segment 3 does not match a booking
                            else:
                                return '0'
                        #the searche is a two segments search and we found at least one booking that match      
                        else:
                            return '1'
                    #segment 2 does not match a booking
                    else:
                        return '0'
                #the searche is a one segment search and we found at least one booking that match
                else:
                    return '1'
            #segment 1 does not match a booking
            else:
                return '0'
     
            
#we create a new dataset from dfEnd with a new colum, 1 if searches match with a booking, 0 if not
dfEnd['booked'] = dfEnd.apply(lambda x: recherche(x), axis=1)
#we export the result in a csv file
dfEnd.to_csv("searches_booked.csv", sep='^')




number of lines in searches.csv :20390199
number of lines in bookings.csv :10000011
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date

Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Colu

Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Colu

Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Colu

Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Colu

...

Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Colu

Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Colu

Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Colu

Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Colu

Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Columns: [cre_date           , brd_port, off_port, brd_time           ]
Index: []
Empty DataFrame
Colu